In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [ ]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [3]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
import random

In [4]:
def load_dataset():
  train_data = pd.read_csv('trainset.csv')
  y_train = pd.read_csv('theta.csv')
  target_data = pd.read_csv('target_data.csv')
  # target_label = pd.read_csv('target_label.csv')
  return train_data, y_train, target_data

In [ ]:
# train_data.head()
# y_train.head()

In [5]:
train_data, y_train, target_data = load_dataset()

In [ ]:
# train_data.shape
# target_data.shape

In [6]:
num_classes = 1
train_data = np.array(train_data.iloc[:,:])
train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
target_data = np.array(target_data.iloc[:,:])
target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [7]:
train_data.shape
target_data.shape

(333, 128, 128)

In [8]:
train_data = train_data.astype('float32')
train_data = train_data/255

rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
rgb_batch.shape

target_data = target_data.astype('float32')
target_data = target_data/255

rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
rgb_target.shape

(333, 128, 128, 3)

In [9]:
target_data = rgb_target

In [10]:
target_data.shape

(333, 128, 128, 3)

In [20]:
y_norm = (2*(y_train - (-25))/(55-(-25))) - 1

In [21]:
print(y_norm)

     theta
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
..     ...
328   -1.0
329   -1.0
330   -1.0
331   -1.0
332   -1.0

[333 rows x 1 columns]


In [23]:
y_train = y_norm

In [24]:
x_train, x_test, y_train, y_test = train_test_split(rgb_batch, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [26]:
input_shape = (128, 128, 3)

In [ ]:
# print(y_test)

     theta  R1  R2   B
3       55   7   0   2
184     15   7 -25  -3
302    -25  -8   0  12
239     -5  -3   0  -3
50      45  -3   0   2
..     ...  ..  ..  ..
14      55  -8   0   2
253     -5   7 -20   2
193      5 -18   0  12
54      45  -3   0  -3
177     15   7 -10   2

[100 rows x 4 columns]


In [27]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(64, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    # model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [28]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [29]:
conv = concatenate([current_model, target_model])

In [30]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=1.0)(dense)
dense = Dropout(0.25)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='tanh')(dense)

model = Model(inputs=[current_input, target_input], outputs=[output])

opt = SGD(learning_rate=0.01)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])


In [ ]:
best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [31]:
model.fit([x_train, target_xtrain], y_train,
          epochs=30,
          validation_data=([x_test, target_xtest], y_test),
          verbose=1)

Epoch 1/30
8/8 [==============================] - 130s 16s/step - loss: 1.1836 - mse: 1.1836 - val_loss: 1.3454 - val_mse: 1.3454
Epoch 2/30
8/8 [==============================] - 119s 15s/step - loss: 1.4649 - mse: 1.4649 - val_loss: 1.3403 - val_mse: 1.3403
Epoch 3/30
8/8 [==============================] - 118s 15s/step - loss: 1.4496 - mse: 1.4496 - val_loss: 1.3172 - val_mse: 1.3172
Epoch 4/30
8/8 [==============================] - 120s 15s/step - loss: 1.4120 - mse: 1.4120 - val_loss: 1.0543 - val_mse: 1.0543
Epoch 5/30
8/8 [==============================] - 125s 16s/step - loss: 0.8901 - mse: 0.8901 - val_loss: 0.4217 - val_mse: 0.4217
Epoch 6/30
8/8 [==============================] - 134s 17s/step - loss: 0.4419 - mse: 0.4419 - val_loss: 0.4087 - val_mse: 0.4087
Epoch 7/30
8/8 [==============================] - 145s 18s/step - loss: 0.4076 - mse: 0.4076 - val_loss: 0.4270 - val_mse: 0.4270
Epoch 8/30
8/8 [==============================] - 136s 17s/step - loss: 0.4730 - mse: 0.47

In [32]:
model.evaluate([x_test, target_xtest], y_test)

4/4 [==============================] - 13s 3s/step - loss: 0.4376 - mse: 0.4376


[0.43758729100227356, 0.43758729100227356]

In [37]:
y_test.head()

,theta
3,1.00
184,0.00
302,-1.00
239,-0.50
50,0.75


In [33]:
prediction = model.predict([x_test, target_xtest])
prediction = np.rint(prediction)

In [35]:
print(prediction)

[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]]


In [38]:
true_value = (((prediction + 1)*(55+25))/2)-25

In [39]:
print(true_value)

[[15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]
 [15.]]


In [40]:
true_gt = (((y_test + 1)*(55+25))/2)-25

In [41]:
print(true_gt)

     theta
3     55.0
184   15.0
302  -25.0
239   -5.0
50    45.0
..     ...
14    55.0
253   -5.0
193    5.0
54    45.0
177   15.0

[100 rows x 1 columns]


In [42]:
arr_gt = true_gt.to_numpy()

In [43]:
print(arr_gt)

[[ 55.]
 [ 15.]
 [-25.]
 [ -5.]
 [ 45.]
 [-15.]
 [ 15.]
 [ 55.]
 [ 25.]
 [-25.]
 [  5.]
 [-25.]
 [ -5.]
 [  5.]
 [  5.]
 [-15.]
 [ -5.]
 [ 15.]
 [ 15.]
 [ 15.]
 [ 15.]
 [-25.]
 [ 45.]
 [ 35.]
 [ 55.]
 [ 55.]
 [  5.]
 [ 35.]
 [  5.]
 [  5.]
 [ 35.]
 [ 15.]
 [ 15.]
 [-15.]
 [ 45.]
 [ 35.]
 [ 55.]
 [ -5.]
 [-15.]
 [ -5.]
 [ 15.]
 [ 25.]
 [-25.]
 [ -5.]
 [ 45.]
 [ 55.]
 [ 15.]
 [-15.]
 [ 55.]
 [ 25.]
 [ 15.]
 [ 15.]
 [ -5.]
 [ 45.]
 [ 55.]
 [ 35.]
 [ 45.]
 [ 15.]
 [ 25.]
 [-25.]
 [ -5.]
 [ 15.]
 [  5.]
 [  5.]
 [-15.]
 [ 55.]
 [ 45.]
 [-25.]
 [  5.]
 [-15.]
 [-15.]
 [ -5.]
 [-15.]
 [ 55.]
 [ 25.]
 [ -5.]
 [-25.]
 [ 35.]
 [ 25.]
 [  5.]
 [ 35.]
 [ 45.]
 [-25.]
 [-15.]
 [ 35.]
 [ 15.]
 [ 35.]
 [-25.]
 [  5.]
 [-25.]
 [ 15.]
 [ 45.]
 [-15.]
 [ 55.]
 [-25.]
 [ 55.]
 [ -5.]
 [  5.]
 [ 45.]
 [ 15.]]


In [46]:
count = 0
for i in range(len(arr_gt)):
  if arr_gt[i]==15:
    count+=1

print(count)

17


In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/data65_27.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img.shape

In [ ]:
img2 = cv2.imread('/content/target_image_1.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

In [ ]:
predictionimg = model.predict([img,img2])

In [ ]:
predictionimg